In [15]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from dotenv import load_dotenv

import os

### Settings Environment

In [16]:
load_dotenv()

PINECONE_KEY = os.getenv('PINECONE_API_KEY')

In [17]:
pc = Pinecone(api_key=PINECONE_KEY)

In [18]:
pc.list_indexes()

{'indexes': []}

### Create Indexes

In [19]:
index_name = "langchain"
if index_name not in pc.list_indexes().names():
    print("Creating index...")
    pc.create_index(
        name="langchain",
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print("Creating Done!!")
else:
    print(f'The index: {index_name} is already exists')

Creating index...
Creating Done!!


Searching Indexes

In [20]:
index_name = "langchain"
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

Delete Indexes

In [37]:
index_name = "langchain"
if index_name in pc.list_indexes().names():
    print("Deleting index...")
    pc.delete_index(index_name)
    print("Deleting Done!!")
else:
    print(f"The index: {index_name} isn't already exists")

Deleting index...
Deleting Done!!


### Work with Vectors

Upsert Vectors

In [22]:
import random

vector = [[random.random() for _ in range(1536)] for i in range(5)]

ids = list("abcde")
index_name = "langchain"

index = pc.Index(index_name)
index.upsert(vectors=zip(ids, vector))

upserted_count: 5

Fetch vector

In [23]:
index.fetch(ids=["c", "d"])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

Delete vector

In [24]:
index.delete(ids=["c", "d"])

In [25]:
index.fetch(ids=["c", "d"])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

Query vector

In [26]:
query = [random.random() for _ in range(1536)]
query[:3]

[0.5472030443138011, 0.08824136968375529, 0.17353503121948954]

In [27]:
index.query(
    vector=query,
    top_k=5,
)

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}

### Namespace

In [28]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

Insert namespace

In [29]:
import random

vector = [[random.random() for _ in range(1536)] for i in range(5)]

ids = list("xyz")
index_name = "langchain"

index = pc.Index(index_name)
index.upsert(vectors=zip(ids, vector), namespace="first-time")

upserted_count: 3

In [30]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}, 'first-time': {'vector_count': 0}},
 'total_vector_count': 0}

In [31]:
import random

vector = [[random.random() for _ in range(1536)] for i in range(5)]

ids = list("opq")
index_name = "langchain"

index = pc.Index(index_name)
index.upsert(vectors=zip(ids, vector), namespace="second-time")

upserted_count: 3

In [32]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3},
                'first-time': {'vector_count': 0},
                'second-time': {'vector_count': 0}},
 'total_vector_count': 3}

Delete namespace

In [33]:
# Don't define the namespace. It's not working because of wrong namespaces
index.delete(ids=["o"])
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3},
                'first-time': {'vector_count': 0},
                'second-time': {'vector_count': 0}},
 'total_vector_count': 3}

In [34]:
index.delete(ids=["o"], namespace="second-time")
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3},
                'first-time': {'vector_count': 3},
                'second-time': {'vector_count': 0}},
 'total_vector_count': 6}

Delete all namespace

In [35]:
index.delete(delete_all=True, namespace="first-time")
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}, 'second-time': {'vector_count': 0}},
 'total_vector_count': 3}

# Create the search similarity

Clear the old index

In [36]:
# for 